In [18]:
from pyomo.environ import *


model = ConcreteModel()

# Set declarations (assuming that the 'orders' and 'machines' sets are based on the maximum values from the data)
model.I = Set(initialize=range(1, 8))  # Set of orders i
model.J = Set(initialize=range(1, 4))  # Set of machines j

# Parameter declarations
# Cost on Machine
model.r = Param(model.I, initialize={1: 2, 2: 3, 3: 4, 4: 5, 5: 10, 6: 1, 7: 2})
model.d = Param(model.I, initialize={1: 16, 2: 13, 3: 21, 4: 28, 5: 24, 6: 28, 7: 23})
model.c = Param(model.I, model.J, initialize={(1, 1): 10, (1, 2): 6, (1, 3): 8,
                                                  (2, 1): 8, (2, 2): 5, (2, 3): 6,
                                                  (3, 1): 12, (3, 2): 7, (3, 3): 10,
                                                  (4, 1): 10, (4, 2): 6, (4, 3): 8,
                                                  (5, 1): 8, (5, 2): 5, (5, 3): 7,
                                                  (6, 1): 12, (6, 2): 7, (6, 3): 10,
                                                  (7, 1): 12, (7, 2): 7, (7, 3): 10})


# Durations p_im for Set 1
model.p = Param(model.I, model.J, initialize={(1, 1): 10, (1, 2): 14, (1, 3): 12,
                                               (2, 1): 6, (2, 2): 8, (2, 3): 7,
                                               (3, 1): 11, (3, 2): 16, (3, 3): 13,
                                               (4, 1): 6, (4, 2): 12, (4, 3): 8,
                                               (5, 1): 10, (5, 2): 16, (5, 3): 12,
                                               (6, 1): 7, (6, 2): 12, (6, 3): 10,
                                               (7, 1): 10, (7, 2): 8, (7, 3): 10}) 


# Uncomment the following and adjust the values for the second set of durations p_im for Set 2
'''
model.p = Param(model.I, model.J, initialize={(1, 1): 5, (1, 2): 7, (1, 3): 6,
                                               (2, 1): 3, (2, 2): 4, (2, 3): 3,
                                               (3, 1): 2, (3, 2): 4, (3, 3): 3,
                                               (4, 1): 3, (4, 2): 6, (4, 3): 4,
                                               (5, 1): 2, (5, 2): 4, (5, 3): 3,
                                               (6, 1): 1, (6, 2): 3, (6, 3): 2,
                                               (7, 1): 1, (7, 2): 2, (7, 3): 1})
'''

'\nmodel.p = Param(model.I, model.J, initialize={(1, 1): 5, (1, 2): 7, (1, 3): 6,\n                                               (2, 1): 3, (2, 2): 4, (2, 3): 3,\n                                               (3, 1): 2, (3, 2): 4, (3, 3): 3,\n                                               (4, 1): 3, (4, 2): 6, (4, 3): 4,\n                                               (5, 1): 2, (5, 2): 4, (5, 3): 3,\n                                               (6, 1): 1, (6, 2): 3, (6, 3): 2,\n                                               (7, 1): 1, (7, 2): 2, (7, 3): 1})\n'

In [19]:
#Define Variables
model.x = Var(model.I,model.J,domain = Binary)
model.ts = Var(model.I,domain = NonNegativeReals)
model.y = Var(Set(initialize=[(i, i_prime) for i in model.I for i_prime in model.I if i != i_prime]),domain = Binary)

In [20]:
#Define constraints

model.earliest_start =  Constraint(model.I, rule=lambda model, i: 
        model.ts[i] >= model.r[i])
model.latest_start =  Constraint(model.I, rule=lambda model, i: 
        model.ts[i] <= model.d[i] - sum(model.p[i,m]*model.x[i,m] for m in model.J))
model.assing_to_one_unit =  Constraint(model.I, rule=lambda model, i: 
        sum(model.x[i,m] for m in model.J)==1)
model.tightening_constraint =  Constraint(model.J, rule=lambda model, m: 
        sum(model.p[i,m]*model.x[i,m] for i in model.I)<=max(model.d[i] for i in model.I)-min(model.r[i] for i in model.I))
model.precedence_constraint =  Constraint(Set(initialize=[(i, i_prime) for i in model.I for i_prime in model.I if i > i_prime]), model.J, rule=lambda model,i,i_prime,m: 
        model.y[i,i_prime]+model.y[i_prime,i] >= model.x[i,m]+model.x[i_prime,m]-1 )
model.M = 1000
model.sequencing_constraint =  Constraint(Set(initialize=[(i, i_prime) for i in model.I for i_prime in model.I if i != i_prime]), rule=lambda model, i,i_prime: 
        model.ts[i_prime] >= model.ts[i]+sum(model.p[i,m]*model.x[i,m] for m in model.J)-model.M*(1-model.y[i,i_prime]))


#model.precedence_constraint2 =  Constraint(Set(initialize=[(i, i_prime,m,m_prime) for i in model.I for i_prime in model.I for m in model.J for m_prime in model.J if i > i_prime and m!=m_prime]),  rule=lambda model,i,i_prime,m,m_prime: 
 #       model.y[i,i_prime]+model.y[i_prime,i]+model.x[i,m]+model.x[i_prime,m_prime] <= 2 )

#Define objective
model.cost = Objective(sense=minimize, expr = sum(model.c[i,m]*model.x[i,m] for i in model.I for m in model.J))



In [21]:
solver = SolverFactory('gurobi')
results = solver.solve(model)
model.pprint()

2 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

4 Param Declarations
    c : Size=21, Index=I*J, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (1, 1) :    10
        (1, 2) :     6
        (1, 3) :     8
        (2, 1) :     8
        (2, 2) :     5
        (2, 3) :     6
        (3, 1) :    12
        (3, 2) :     7
        (3, 3) :    10
        (4, 1) :    10
        (4, 2) :     6
        (4, 3) :     8
        (5, 1) :     8
        (5, 2) :     5
        (5, 3) :     7
        (6, 1) :    12
        (6, 2) :     7
        (6, 3) :    10
        (7, 1) :    12
        (7, 2) :     7
        (7, 3) :    10
    d : Size=7, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
    

In [22]:
import pandas as pd

# Define column names
#columns = [1, 2, 3]

# Create an empty DataFrame
df_xsol = pd.DataFrame(columns=model.J, index=model.I)
df_ysol = pd.DataFrame(columns=model.I, index=model.I)
for i in model.I:
    for j in model.J:
        df_xsol.at[i,j] = model.x[i,j]()
for i in model.I:
    for j in model.I:
        if(i!=j):
            df_ysol.at[i,j] = model.y[i,j]()
print(df_xsol)
print(df_ysol)
print(model.cost())

     1    2    3
1  1.0 -0.0 -0.0
2 -0.0  1.0 -0.0
3 -0.0 -0.0  1.0
4  1.0 -0.0 -0.0
5  1.0  0.0 -0.0
6 -0.0 -0.0  1.0
7 -0.0  1.0 -0.0
     1    2    3    4    5    6    7
1  NaN  0.0  0.0  1.0  1.0  0.0  0.0
2  0.0  NaN -0.0  0.0  0.0  0.0  1.0
3  0.0  0.0  NaN  0.0  0.0  1.0  0.0
4  0.0  0.0  0.0  NaN -0.0 -0.0 -0.0
5  0.0  0.0  0.0  1.0  NaN -0.0  0.0
6  0.0  0.0 -0.0 -0.0 -0.0  NaN -0.0
7  0.0  0.0  0.0  0.0 -0.0 -0.0  NaN
60.0
